In [6]:
import os
from pysar.objects import sensor

work_dir = '~/insarlab/PYSAR_EXAMPLE/AlcedoSenDT128/PYSAR'
work_dir = '~/insarlab/PYSAR_EXAMPLE/SierraNegraAlosAT133/PYSAR'

work_dir = os.path.expanduser(work_dir)
os.chdir(work_dir)
print('Go to directory: '+work_dir)

proj_name = sensor.project_name2sensor_name(work_dir)[1]

plotCmd = 'view.py avgPhaseVelocity.h5 -m maskSpatialCoh.h5 '
plotCmd += '--fontsize 12 --figsize 4 3 --notitle --dpi 600 --nodisplay --cbar-label "Velocity [cm/yr]" '
plotCmd += '-o {}_avgPhaseVel.jpg'.format(proj_name)
print(plotCmd)
os.system(plotCmd)


Go to directory: /Users/yunjunz/insarlab/PYSAR_EXAMPLE/SierraNegraAlosAT133/PYSAR
view.py avgPhaseVelocity.h5 -m maskSpatialCoh.h5 --fontsize 12 --figsize 4 3 --notitle --dpi 600 --nodisplay --cbar-label "Velocity [cm/yr]" -o SierraNegraAlosAT133_avgPhaseVel.jpg


0